In [ ]:
import kagglegym
import numpy as np
import pandas as pd
import math
import time
import gc
from sklearn.base import BaseEstimator, clone
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

class FastBaggingRegressor(BaseEstimator):
    def __init__(self, base_estimator=None, n_estimators=10, random_state=123456,
                 max_samples=1.0, max_features=0, bootstrap=True, bootstrap_features=False, filter_estimators=False):
        self.base_estimator = base_estimator
        self.n_estimators = n_estimators
        self.max_samples = max_samples
        self.max_features = max_features
        self.bootstrap = bootstrap
        self.bootstrap_features = bootstrap_features
        self.filter_estimators = filter_estimators
        self.is_fitted = False
        self.model_coef = np.zeros((n_estimators,))  # np.array(n_estimators, 1)
        self.model_constants = np.zeros((n_estimators,))  # np.array(n_estimators, 1)
        self.model_rewards = np.zeros((n_estimators,))  # np.array(n_estimators, 1)
        self.n_features = None
        self.prng = np.random.RandomState(random_state)

    def normalized_rms_error(self, y_true, y_pred):
        return np.sum((y_true - y_pred)**2) / np.sum((y_true - np.mean(y_true))**2)

    def fit(self, X, y):
        try:
            X = X.values
            y = y.values
        except AttributeError:
            pass

        n_samples, self.n_features = X.shape
        self.model_coef = np.zeros((self.n_estimators, self.n_features))

        assert self.base_estimator is not None, 'please set base_estimator'

        features_indices_ = np.arange(self.n_features)
        sample_indices_ = np.arange(n_samples)
        train_filter = np.zeros((n_samples,), dtype=bool)
        current_model = clone(self.base_estimator)

        for idx_ in range(self.n_estimators):
            train_filter[:] = False
            train_indices = self.prng.choice(sample_indices_, size=int(self.max_samples * n_samples), replace=True)
            train_filter[train_indices] = True

            if self.bootstrap_features:
                n_selected_features = int(self.max_features)
                current_features = self.prng.choice(features_indices_, size=n_selected_features, replace=False)
            else:
                current_features = features_indices_

            current_model.fit(X[train_filter, current_features], y[train_filter])
            y_pred = current_model.predict(X[~train_filter, current_features])

            self.model_rewards[idx_] = self.normalized_rms_error(y[~train_filter], y_pred)
            self.model_constants[idx_] = current_model.intercept_
            self.model_coef[idx_, current_features] = current_model.coef_

            if idx_ % 50 == 0:
                print('FastBaggingRegressor finished %d out of %d' % (idx_, self.n_estimators))

        self.final_model = clone(self.base_estimator)
        if self.filter_estimators:
            selected_models = self.model_rewards.argsort()[-int(0.5 * self.n_estimators):]
            self.final_model.coef_ = np.mean(self.model_coef[selected_models, :], axis=0)
            self.final_model.intercept_ = np.mean(self.model_constants[selected_models], axis=0)
        else:
            self.final_model.coef_ = np.mean(self.model_coef, axis=0)
            self.final_model.intercept_ = np.mean(self.model_constants, axis=0)

        self.is_fitted = True
        self.coef_ = self.final_model.coef_
        self.intercept_ = self.final_model.intercept_
        return self

    def predict(self, X):
        if not self.is_fitted:
            raise RuntimeError('Model is not fitted yet')
        try:
            X = X.values
        except AttributeError:
            pass
        return self.final_model.predict(X)


def _reward(y_true, y_fit):
    R2 = 1 - np.sum((y_true - y_fit)**2) / np.sum((y_true - np.mean(y_true))**2)
    R = np.sign(R2) * math.sqrt(abs(R2))
    return R


def index_chunker(data_df, n_chunks):
    unique_timestamp = data_df["timestamp"].unique()
    n_total = len(unique_timestamp)
    chunk_size = int(n_total / n_chunks)
    chunk_boundaries = np.arange(0, n_total, chunk_size)
    index_df = pd.DataFrame(data=True, index=data_df.index, columns=range(chunk_boundaries.shape[0]))

    for idx_, pos in enumerate(chunk_boundaries):
        if (pos + 2 * chunk_size) > n_total:
            current_filter = (data_df.timestamp >= pos)
            index_df.loc[current_filter, idx_] = False

            if (idx_ + 1) <= index_df.shape[0]:
                index_df = index_df.iloc[:, :(idx_ + 1)]
            break

        current_filter = (data_df.timestamp >= pos) & (data_df.timestamp < (pos + chunk_size))
        index_df.loc[current_filter, idx_] = False

    return index_df


def train_base_model_cv(train, y_train, model_columns, sk_model, predictor_name):
    if predictor_name not in train.columns:
        train.assign(**{predictor_name: np.nan})

    train_columns = ['timestamp'] + model_columns
    X_train = train[train_columns]

    n_chunks = 10
    train_test_df = index_chunker(X_train, n_chunks)

    for column_idx_ in train_test_df.columns:
        train_indices = train_test_df[column_idx_]
        sk_model.fit(X_train.loc[train_indices, model_columns], y_train[train_indices])
        train.loc[~train_indices, predictor_name] = sk_model.predict(X_train.loc[~train_indices, model_columns])

    X_train = train[model_columns]
    sk_model.fit(X_train, y_train)

    return sk_model


def process_features_training(train, id_mean_columns_list, id_mean_rolling_list):
    print('Processing features during training')

    train['tech23'] = train['technical_20'] + train['technical_13'] - train['technical_30']
    train['tech23_v2'] = train['technical_20'] - train['technical_30']

    grouped_data_id = train.groupby('id')

    process_tech23 = lambda x: (x - 0.925 * x.shift(1)) / 0.075
    approx_y = grouped_data_id['tech23'].transform(process_tech23)
    train['approx_y_prev'] = approx_y.fillna(0).clip(-0.07, 0.07)

    approx_y = grouped_data_id['tech23_v2'].transform(process_tech23)
    train['approx_y_prev_v2'] = approx_y.fillna(0).clip(-0.07, 0.07)

    for current_column in id_mean_columns_list:
        for n_rolling_length in id_mean_rolling_list:
            temp_data = grouped_data_id[current_column].rolling(window=n_rolling_length).mean().fillna(0.0)
            temp_data = pd.DataFrame(temp_data.values, index=temp_data.index.get_level_values(1))

            new_column_name1 = f"{current_column}_id_mean_{n_rolling_length}"
            train[new_column_name1] = temp_data

            new_column_name2 = f"{current_column}_id_diff_mean_{n_rolling_length}"
            train[new_column_name2] = train[current_column] - train[new_column_name1]

    grouped_data_ts = train.groupby('timestamp')
    average_values = grouped_data_ts['tech23'].agg([np.mean, np.std])
    average_values_extended = average_values.loc[train['timestamp']].reset_index(drop=True)
    train['tech23_cs_mean'] = average_values_extended['mean'].values
    train['tech23_cs_std'] = average_values_extended['std'].values

    n_rolling_length_long = 10
    average_values = grouped_data_ts['tech23'].mean().rolling(n_rolling_length_long).mean().fillna(0.0)
    train['tech23_cs_mean_10'] = average_values.loc[train['timestamp']].reset_index(drop=True).values

    train_median = train.median(axis=0)
    train = train.replace([np.inf, -np.inf], np.nan).fillna(train_median)

    return train, train_median


def process_features_online(test, id_mean_columns_list, id_mean_rolling_list, prev_test_lists, train_median):
    print('Processing features during prediction')

    max_rolling_length = np.max(id_mean_rolling_list)

    test['tech23'] = test['technical_20'] + test['technical_13'] - test['technical_30']
    test['tech23_cs_mean'] = test['tech23'].mean(axis=0)
    test['tech23_cs_std'] = test['tech23'].std(axis=0)

    test_prev = prev_test_lists[-1]
    approx_y_prev_values = test[['id', 'tech23']].set_index('id') - 0.925 * (
                test_prev[['id', 'tech23']].set_index('id'))
    approx_y_prev_values /= 0.075

    test['approx_y_prev'] = approx_y_prev_values.loc[test.id].values
    test['approx_y_prev'].fillna(0, inplace=True)

    test['tech23_v2'] = test['technical_20'] - test['technical_30']
    approx_y_prev_values = test[['id', 'tech23_v2']].set_index('id') - 0.925 * (
                test_prev[['id', 'tech23_v2']].set_index('id'))
    approx_y_prev_values /= 0.075

    test['approx_y_prev_v2'] = approx_y_prev_values.loc[test.id].values
    test['approx_y_prev_v2'].fillna(0, inplace=True)

    prev_test_lists.append(test)
    if len(prev_test_lists) > max_rolling_length:
        prev_test_lists = prev_test_lists[1:]

    test_temp = test.set_index(test.id)
    for n_rolling_length in id_mean_rolling_list:
        test_grouped_data_id = pd.concat(prev_test_lists[-n_rolling_length:], axis=0).groupby('id')
        for current_column in id_mean_columns_list:
            if current_column == 'approx_y_prev':
                temp_data = test_grouped_data_id[current_column].median()
            else:
                temp_data = test_grouped_data_id[current_column].mean()

            new_column_name1 = f"{current_column}_id_mean_{n_rolling_length}"
            test_temp[new_column_name1] = temp_data

            new_column_name2 = f"{current_column}_id_diff_mean_{n_rolling_length}"
            test_temp[new_column_name2] = test_temp[current_column] - test_temp[new_column_name1]

    test_grouped_data_id = pd.concat(prev_test_lists, axis=0).groupby('id')
    test_temp['tech23_cs_mean_10'] = test_grouped_data_id['tech23_cs_mean'].mean()

    test = test_temp.reset_index(drop=True)
    test = test.fillna(train_median)

    return test, prev_test_lists


env = kagglegym.make()
observation = env.reset()

train = observation.train

id_mean_columns_list = ['tech23', 'approx_y_prev', 'tech23_v2', 'approx_y_prev_v2']
id_mean_rolling_list = [3, 5, 7, 10]
train, train_median = process_features_training(train, id_mean_columns_list, id_mean_rolling_list)

low_y_cut = -0.085
high_y_cut = 0.092

mean_normalization_column = 'tech23_cs_mean_10'
volatility_column = 'tech23_cs_std'
cs_factor = 0.25

y_train = train['y'] - cs_factor * train[mean_normalization_column]
y_train /= train[volatility_column]

base_models = []
simple_model = make_pipeline(StandardScaler(), LinearRegression(fit_intercept=True))

base_models_columns = [['tech23', 'tech23_id_mean_3'],
                       ['technical_40', 'fundamental_11'],
                       ['approx_y_prev', 'approx_y_prev_id_mean_3', 'approx_y_prev_id_mean_10'],
                       # ... (add more base models as needed)
                       ]

n_base_models = len(base_models_columns)

for idx_ in range(n_base_models):
    model_columns_ = base_models_columns[idx_]
    predictor_name = f'pred_{idx_}'
    sk_model_ = clone(simple_model)
    sk_model_ = train_base_model_cv(train, y_train, model_columns_, sk_model_, predictor_name)
    base_models.append(sk_model_)

base_models_outputs = [f'pred_{idx_}' for idx_ in range(n_base_models)]
model_final_columns = base_models_outputs

X_train = train[model_final_columns]
predictor_stacked_name = 'stacked_pred'

sk_model_final = FastBaggingRegressor(LinearRegression(fit_intercept=True), n_estimators=3000,
                                      random_state=565776, max_samples=1.0, max_features=3,
                                      bootstrap=True, bootstrap_features=True, filter_estimators=True)

sk_model_final.fit(X_train, y_train)

print(f'sk_model_final with parameters intercept_ and coef_: {sk_model_final.intercept_}, {sk_model_final.coef_}')

max_rolling_length = np.max(id_mean_rolling_list)
prev_test_lists = list(train.groupby('timestamp'))[-max_rolling_length:]
prev_test_lists = [x[1] for x in prev_test_lists]

i = 0
reward_ = []

while True:
    test = observation.features
    pred = observation.target

    test, prev_test_lists = process_features_online(test, id_mean_columns_list, id_mean_rolling_list,
                                                    prev_test_lists, train_median)

    for temp_pred_name in model_final_columns + [predictor_stacked_name]:
        if temp_pred_name not in test.columns:
            test[temp_pred_name] = np.nan

    for idx_local_ in range(n_base_models):
        model_columns_ = base_models_columns[idx_local_]
        predictor_name = f'pred_{idx_local_}'
        sk_model_ = base_models[idx_local_]
        X_current = test[model_columns_]
        test[predictor_name] = sk_model_.predict(X_current)

    X_current = test[model_final_columns]
    test[predictor_stacked_name] = sk_model_final.predict(X_current)

    test[predictor_stacked_name].replace([np.inf, -np.inf, np.nan], 0.0, inplace=True)

    test['y_pred'] = test[predictor_stacked_name]
    test['y_pred'] *= test[volatility_column]
    test['y_pred'] += cs_factor * test[mean_normalization_column]

    test['y_pred'] = test['y_pred'].clip(low_y_cut, high_y_cut)

    pred['y'] = test['y_pred']
    observation, reward, done, info = env.step(pred[['id', 'y']])
    reward_.append(reward)

    if i % 100 == 0:
        print(reward, np.mean(np.array(reward_)))
        collected = gc.collect()
        print(f'Garbage collector: collected {collected} objects.')

    i += 1

    if done:
        print(f'Finished ... {info["public_score"]}')
        break